In [32]:
!pip install -q groq 



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
!pip install -q langchain 



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
!pip install -q faiss-cpu

^C



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
!pip install -q sentence-transformers


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
!pip install -q pypdf 


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
!pip install -q python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
!pip install -q  ipywidgets


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
from typing import List, Optional
from IPython.display import display, Markdown
import ipywidgets as widgets
from dotenv import load_dotenv
from groq import Groq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
load_dotenv()  # Load environment variables from .env file

class Config:
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    DATA_PATH = "data/"  
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 200
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    LLM_MODEL = "llama3-70b-8192"

In [14]:
try:
    client = Groq(api_key=Config.GROQ_API_KEY)
    display(Markdown("✅ **Groq client initialized successfully**"))
except Exception as e:
    display(Markdown(f"❌ **Error initializing Groq client:** {e}"))

✅ **Groq client initialized successfully**

In [2]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [3]:
import os

In [ ]:
api_key_input = widgets.Text(
    placeholder='Enter your Groq API key',
    description='API Key:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(
    description="Submit",
    button_style='success'
)

output_area = widgets.Output()

# Create help text
help_text = widgets.HTML(
    value="<small>Get your API key from <a href='https://console.groq.com/keys' target='_blank'>Groq Console</a></small>"
)

# Create container
input_container = widgets.VBox([
    widgets.HTML("<h3>Enter your Groq API key</h3>"),
    api_key_input,
    help_text,
    submit_button,
    output_area
])

display(input_container)

def on_submit_clicked(b):
    with output_area:
        clear_output()
        api_key = api_key_input.value.strip()
        
        if not api_key:
            display(Markdown("❌ **Please enter a valid API key**"))
            return
        
        try:
            from groq import Groq
            test_client = Groq(api_key=api_key)
            test_client.models.list()  
            
            os.environ["GROQ_API_KEY"] = api_key
            api_key_input.disabled = True
            submit_button.disabled = True
            
            display(Markdown("✅ **API key validated successfully!**"))
            display(Markdown("You can now proceed to run the next cells."))
            
        except Exception as e:
            display(Markdown(f"❌ **Key verification failed:** {str(e)}"))
            if "GROQ_API_KEY" in os.environ:
                del os.environ["GROQ_API_KEY"]

submit_button.on_click(on_submit_clicked)



if 'GROQ_API_KEY' not in os.environ:
    display(Markdown("❌ **API key not configured. Please complete Step 1 first.**"))
else:
    try:
        from groq import Groq
        client = Groq(api_key=os.environ["GROQ_API_KEY"])
        
        display(Markdown("✅ **Chatbot components initialized successfully!**"))
        
        next_step_button = widgets.Button(
            description="Continue to Document Processing",
            button_style='info'
        )
        
        def on_next_step_clicked(b):
            display(Markdown("Proceeding to document processing..."))
           
            
        next_step_button.on_click(on_next_step_clicked)
        display(next_step_button)
        
    except Exception as e:
        display(Markdown(f"❌ **Initialization failed:** {str(e)}"))

✅ **Chatbot components initialized successfully!**

Button(button_style='info', description='Continue to Document Processing', style=ButtonStyle())

In [9]:
extracted_data = load_pdf("data/")


In [16]:
!pip install langchain langchain-google-genai langchain-community pypdf


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from typing import List

In [ ]:
def document_processing_status():
    """Show document processing status with widgets"""
    progress = widgets.IntProgress(value=0, min=0, max=4, description='Processing:')
    status_label = widgets.Label(value="Waiting to start...")
    display(widgets.VBox([progress, status_label]))
    return progress, status_label

def load_and_process_documents():
    """Load and process documents with visual feedback"""
    progress, status = document_processing_status()
    
    try:
        if not os.path.exists(Config.DATA_PATH):
            os.makedirs(Config.DATA_PATH, exist_ok=True)
            status.value = f"Created directory {Config.DATA_PATH}. Please add PDF files and run again."
            return None
            
        status.value = "Loading PDF documents..."
        progress.value = 1
        
        loader = DirectoryLoader(Config.DATA_PATH, glob="*.pdf", loader_cls=PyPDFLoader)
        documents = loader.load()
        
        if not documents:
            status.value = f"No PDFs found in {Config.DATA_PATH}. Please add documents."
            return None
            
        status.value = f"Splitting {len(documents)} documents into chunks..."
        progress.value = 2
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=Config.CHUNK_SIZE,
            chunk_overlap=Config.CHUNK_OVERLAP,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )
        chunks = text_splitter.split_documents(documents)
        
        status.value = "Creating embeddings (this may take a minute)..."
        progress.value = 3
        
        embeddings = HuggingFaceEmbeddings(
            model_name=Config.EMBEDDING_MODEL,
            model_kwargs={'device': 'cpu'},  # Use 'cuda' if you have GPU
            encode_kwargs={'normalize_embeddings': False}
        )
        
        status.value = "Building vector database..."
        progress.value = 4
        
        vector_store = FAISS.from_documents(chunks, embeddings)
        
        status.value = f"✅ Success! Processed {len(documents)} docs into {len(chunks)} chunks"
        return vector_store
        
    except Exception as e:
        status.value = f"❌ Error: {str(e)}"
        return None

In [ ]:
display(Markdown("### Document Processing"))
vector_store = load_and_process_documents()



class MedicalChatbot:
    def __init__(self, vector_store=None):
        self.vector_store = vector_store
        self.system_prompt = """You are Dr. Bot, a medical AI assistant. Follow these rules:
1. Provide accurate, evidence-based information
2. Explain medical concepts in simple terms
3. Never diagnose or prescribe treatment
4. Always recommend consulting a doctor
5. Be empathetic and professional"""
        
        self.setup_ui()
        
    def setup_ui(self):
        """Initialize the user interface components"""
        # Chat history display
        self.chat_history = widgets.Output()
        display(self.chat_history)
        
        self.user_input = widgets.Text(
            placeholder='Type your medical question here...',
            layout=widgets.Layout(width='80%')
        )
        
        self.submit_button = widgets.Button(
            description="Ask",
            button_style='primary',
            tooltip='Submit your question'
        )
        self.submit_button.on_click(self.process_input)
        
        # Clear button
        self.clear_button = widgets.Button(
            description="Clear",
            button_style='warning'
        )
        self.clear_button.on_click(self.clear_chat)
        
        # Display controls
        display(widgets.HBox([self.user_input, self.submit_button, self.clear_button]))
    
    def get_context(self, query: str, k: int = 3) -> str:
        """Retrieve relevant context from documents"""
        if not self.vector_store:
            return ""
        try:
            docs = self.vector_store.similarity_search(query, k=k)
            return "\n\n".join([f"Source {i+1}:\n{doc.page_content}" 
                              for i, doc in enumerate(docs)])
        except Exception as e:
            print(f"Context retrieval error: {e}")
            return ""
    
    def generate_response(self, query: str) -> str:
        """Generate response using Groq API"""
        context = self.get_context(query)
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ]
        
        try:
            response = client.chat.completions.create(
                model=Config.LLM_MODEL,
                messages=messages,
                temperature=0.3,
                max_tokens=1024
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"I'm having trouble responding. Error: {str(e)}"
    
    def process_input(self, _):
        """Handle user input"""
        query = self.user_input.value.strip()
        if not query:
            return
            
        with self.chat_history:
            display(Markdown(f"**You:** {query}"))
            
            # Show typing indicator
            with widgets.Output():
                display(Markdown("*Dr. Bot is thinking...*"))
            
            response = self.generate_response(query)
            
            # Clear typing indicator and show response
            clear_output(wait=True)
            display(Markdown(f"**Dr. Bot:** {response}"))
        
        self.user_input.value = ""
    
    def clear_chat(self, _):
        """Clear the chat history"""
        self.chat_history.clear_output()

if 'client' in globals() and vector_store is not None:
    display(Markdown("## 🏥 Medical Chatbot Interface"))
    chatbot = MedicalChatbot(vector_store)
else:
    display(Markdown("❌ Chatbot initialization failed. Please check:"
                    "\n1. Your API key is valid"
                    "\n2. You have PDF files in the data directory"
                    "\n3. All previous cells ran successfully"))

### Document Processing

## 🏥 Medical Chatbot Interface

Output()